# Finite Element Advection Diffusion Equation

## Weak formulation

We will start with the 2D pure conduction equation:
$$\rho c \frac{\partial T}{\partial t} = \frac{\partial}{\partial x}\biggl(k \frac{\partial T}{\partial x}\biggr) + \frac{\partial}{\partial y}\biggl(k \frac{\partial T}{\partial y}\biggr)$$
where $k$ is the material conductivity, $\rho$ is the material density, $c$ is the material capacitance.  

Let's multiply each side of the equation on the test function (shape function) $\phi$ and integrate it over the domain volume $\Omega$):
$$
\int_{\Omega}{\rho c \frac{\partial T}{\partial t} \phi_e dx dy} = \int_{\Omega}{\frac{\partial}{\partial x}\biggl(k \frac{\partial{T}}{\partial x} \biggr)\phi_e dx dy} + \int_{\Omega}{\frac{\partial}{\partial y}\biggl(k \frac{\partial{T}}{\partial y}\biggr) \phi_e dx dy} 
$$

We need to use Gauss-Ostrogradsky theorem which is represented as:
$$\int_{\Omega}{\mathbf{\nabla}\cdot \mathbf{F} dV} = \int_{\partial\Omega}{\mathbf{F}\cdot\mathbf{n}} dS$$

Notice also, that the following can be represented as:
$$\mathbf{\nabla}\cdot \phi \mathbf{\nabla} G = \phi \mathbf{\nabla}\cdot\mathbf{\nabla G} + \mathbf{\nabla}\phi\cdot\mathbf{\nabla}G$$


Assuming that $k$, $\rho$ and $c$ are all constants and $G=k T$, the following is obtained:
$$
\int_{\Omega}{\rho c \partial_t T \phi dV} =\int_{\partial \Omega}{k\partial_n T \phi dS} - \int_{\Omega}{\biggl(k \partial_x T \partial_x \phi + k \partial_y T \partial_y \phi\biggr) dV}
$$

We now need to take integrals over the boundary. For the Newmann boundary condition there is a prescribed flux boundary condition: $k\partial_n T|_{\partial \Omega_N} = g_N$. Whenever on boundary we have a prescribed Dirichlet boundary condition, i.e. $T|_{\partial \Omega_D} = g_D$, to avoid calculating the integral over the boundary $\int_{\partial \Omega}{k\partial_n T\phi dS}$ for Dirichlet boundary condition we ask a test function to have a zero value on the boudnary $\phi|_{\partial \Omega_D} = 0$.

Therefore the following integral can be calculated as follows:
$$
\int_{\Omega}{\rho c \partial_t T \phi dV} =\int_{\partial \Omega_N}{g_N \phi dS} - \int_{\Omega}{\biggl(k \partial_x T \partial_x \phi + k \partial_y T \partial_y \phi\biggr) dV}
$$

Now we approximate the test function $\phi$ from the space of the first order polynomials defined in the whole domain. We also approximate the temperature function as the summation through all possible first order polynomials. However, it's almost impossible to work with all possible first order polynomials in space and represent the temperature through their summation ($\sum_{i=0}^{\infty}{a_i x + b_i y + c_i}$). 

Instead, we introduce the discretization of space into certain geometrical features, say the domain $\Omega$ is subdivided onto $M$ features (triangles in our case) and have $N$ nodes. 

In each geometrical feature $ e = 1..M$ the solution is represented through one of polynomials. Assuming that we have a basis for polynomials in this geometrical feature, one can write:  
$$
T_e = \sum_{j=1}^{d}{a_j \phi^e_j(x,y)},
$$
where $a_j$ are the weights, $d$ is the number of coefficients, $\phi_j^e$ is the linear shape function corresponding to the node $j$ of the element $e$. 

A few things to note in triangle we have only three points that will be searched for in the final solution, so we will have only three basis functions inside the triangle.  At this particular point nothing stops us to assume that the shape function takes $1$ at node itself and $0$ at any other nodes. So the profile of temperatures inside the geometrical feature $e$ can be represented through its nodal values:
$$
T_e = \sum_{j=1}^{d}{T_j^e \phi^e_j(x,y)},
$$
where $\phi^e_j(x_j,y_j) = 1$ and $\phi^e_j(x_k,y_k)=0$ for $j\neq k$ ($x_j,y_j$ are coordinates of the node $j$). Also, outside the element $e$ this shape function equals zero. Notice, that each shape function is represented as $\phi^e_j(x,y) = a_j x + b_j y +c_j$.

Now the big integral (assuming that the basis function has non-zero value only inside the corresponding triangle), then the whole integral is taken as summation over the geometrical features. If we take the test function to be a shape function, then the integral with this shape function $\phi^e_j$ will be taken only inside the corresponding geometrical feature:
$$
\int_{\Omega^e}{\rho c \partial_t \sum_k{T_k^e \phi^e_k} \phi^e_j dV}=\int_{\partial\Omega_N^e}{g^e_N \phi_j^e dS} - \int_{\Omega^e}{\biggl(k\partial_x \phi_j^e \sum_k{T_k^e \partial_x \phi_k^e + k\partial_y \phi_j^e \sum_k{T_k^e \partial_y \phi_k^e}}\biggr)}
$$

We discretize in time using the implicit discretization:
$$
\partial_t T_k^e = \frac{{T_k^e}_{n+1}-{T_k^e}_n}{t_{n+1}-t_n}
$$


Eventually we will have the linear system involving the nodal temperatures. Let us see derive this matrix:
$$
\int_{\Omega^e}{\frac{\rho c}{t_{n+1}-t_n}  \sum_k{\bigl({T_k^e}_{n+1}-{T_k^e}_n\bigr) \phi^e_k} \phi^e_j dV}=\int_{\partial\Omega_N^e}{{g^e_N}_{n+1} \phi_j^e dS} - \int_{\Omega^e}{\biggl(k\partial_x \phi_j^e \sum_k{{T_k^e}_{n+1} \partial_x \phi_k^e + k\partial_y \phi_j^e \sum_k{{T_k^e}_{n+1} \partial_y \phi_k^e}}\biggr)}
$$

For each element we have the following coefficients:
$$
\frac{\rho c}{t_{n+1}-t_n}\sum_k{{T_k^e}_{n+1} M_{kj}}-\frac{\rho c}{t_{n+1}-t_n}\sum_k{{T_k^e}_n M_{kj}} = 
\int_{\partial_{\Omega^e_N}}{{g^e_N}_{n+1}\phi^e_j dS} - k\sum_k{{T_k^e}_{n+1} A_{kj}},
$$
where the mass matrix is $M_{kj} = \int_{\Omega^e}{\phi_k^e\phi_j^e dV}$ and the stiffness matrix $A_{kj} = \int_{\Omega^e}{\bigl(\partial_x \phi_k^e \partial_x\phi_j^e +\partial_y \phi_k^e \partial_y\phi_j^e \bigr)}$.

Overall we can obtain the matrix equation:
$$
\frac{\rho c}{t_{n+1}-t_n} \mathbf{M} \mathbf{T}_{n+1} + k \mathbf{A} T_{n+1} = \mathbf{G}_{n+1}+\frac{\rho c}{t_{n+1}-t_n} \mathbf{M}\mathbf{T}_n 
$$

This equation is valid only for one element but eventually we will have many elements contributing to the same node. It means that the contributions need to be summed up.

# Shape functions

So far we didn't talk much about the shape functions. Those need to equal $1$ on the corresponding node and $0$ on another one. Given arbitrary coordinates of the triangle, we kind of need to come up with the new shape functions formulation for each new triangle. Instead, we will offer the basic form of the shape functions.

Each triangle with coordinates of vertices $(x_0,y_0)$, $(x_1,y_1)$ and $(x_2,y_2)$ is transformed to the basis triangle with coordinates  $(0,0)$, $(1,0)$ and $(1,1)$. 

<img src="mapping.png"/>

Let's derive the transformation matrix:
$$
\begin{pmatrix}
0\\
0
\end{pmatrix} = 
A 
\begin{pmatrix}
x_0\\
y_0\\
\end{pmatrix} + B
$$
$$
\begin{pmatrix}
1\\
0
\end{pmatrix} = 
A 
\begin{pmatrix}
x_1\\
y_1\\
\end{pmatrix} + B
$$
$$
\begin{pmatrix}
0\\
1
\end{pmatrix} = 
A 
\begin{pmatrix}
x_2\\
y_2\\
\end{pmatrix} + B
$$



In [2]:
import sympy

In [4]:
x0,y0,x1,y1,x2,y2 = sympy.symbols('x0,y0,x1,y1,x2,y2')

Matrix $A$ and vector $B$ represent the transformation:
$$
A = 
\begin{pmatrix}
a&b\\
c&d\\
\end{pmatrix}
$$
$$
B=
\begin{pmatrix}
e\\
f
\end{pmatrix}
$$

In terms of unknown things, i.e. $a$,$b$,$c$,$d$,$e$,$f$, we can write the following:
$$
\begin{pmatrix}
0\\
0\\
1\\
0\\
0\\
1
\end{pmatrix} = 
\begin{pmatrix}
x_0&y_0&1&0&0&0\\
0&0&0&x_0&y_0&1\\
x_1&y_1&1&0&0&0\\
0&0&0&x_1&y_1&1\\
x_2&y_2&1&0&0&0\\
0&0&0&x_2&y_2&1\\
\end{pmatrix}
\begin{pmatrix}
a\\
b\\
e\\
c\\
d\\
f
\end{pmatrix}
$$

In [12]:
a,b,c,d,e,f = sympy.symbols('a,b,c,d,e,f')
system = sympy.Matrix(((x0,y0,1,0,0,0,0),(0,0,0,x0,y0,1,0),(x1,y1,1,0,0,0,1),(0,0,0,x1,y1,1,0),(x2,y2,1,0,0,0,0),(0,0,0,x2,y2,1,1)))

In [20]:
solution=sympy.solve_linear_system(system,a,b,e,c,d,f)
solution

{a: (-y0 + y2)/(x0*y1 - x0*y2 - x1*y0 + x1*y2 + x2*y0 - x2*y1),
 b: (x0 - x2)/(x0*y1 - x0*y2 - x1*y0 + x1*y2 + x2*y0 - x2*y1),
 e: (-x0*y2 + x2*y0)/(x0*y1 - x0*y2 - x1*y0 + x1*y2 + x2*y0 - x2*y1),
 c: (y0 - y1)/(x0*y1 - x0*y2 - x1*y0 + x1*y2 + x2*y0 - x2*y1),
 d: (-x0 + x1)/(x0*y1 - x0*y2 - x1*y0 + x1*y2 + x2*y0 - x2*y1),
 f: (x0*y1 - x1*y0)/(x0*y1 - x0*y2 - x1*y0 + x1*y2 + x2*y0 - x2*y1)}

Let's check if we obtained equations right:

In [25]:
print(sympy.simplify(solution[a]*x0+solution[b]*y0+solution[e]))
print(sympy.simplify(solution[c]*x0+solution[d]*y0+solution[f]))
print(sympy.simplify(solution[a]*x1+solution[b]*y1+solution[e]))
print(sympy.simplify(solution[c]*x1+solution[d]*y1+solution[f]))
print(sympy.simplify(solution[a]*x2+solution[b]*y2+solution[e]))
print(sympy.simplify(solution[c]*x2+solution[d]*y2+solution[f]))

0
0
1
0
0
1


Therefore the transformation is as follows:
$$
\begin{aligned}
s =\frac{-y_0 + y_2}{x_0 y_1 - x_0 y_2 - x_1 y_0 + x_1 y_2 + x_2 y_0 - x_2 y_1} x + \frac{x0 - x2}{x_0 y_1 - x_0 y_2 - x_1 y_0 + x_1 y_2 + x_2 y_0 - x_2 y_1} y + \frac{-x_0 y_2 + x_2 y_0}{x_0 y_1 - x_0 y_2 - x_1 y_0 + x_1 y_2 + x_2 y_0 - x_2 y_1}\\
t = \frac{y0 - y1}{x_0 y_1 - x_0 y_2 - x_1 y_0 + x_1 y_2 + x_2 y_0 - x_2 y_1} x + \frac{-x0 + x1}{x_0 y_1 - x_0 y_2 - x_1 y_0 + x_1 y_2 + x_2 y_0 - x_2 y_1} y + \frac{x_0 y_1 - x_1 y_0}{x_0 y_1 - x_0 y_2 - x_1 y_0 + x_1 y_2 + x_2 y_0 - x_2 y_1}
\end{aligned}
$$

Now let's 